In [1]:
%%time

import tensorflow as tf
print(tf.__version__)
import sys
sys.path.append("..")
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


def plot_loss(history, *losses):
    for loss in losses:
        plt.plot(history.history[loss], label=loss)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

def scaling(x, min, max):
    return np.where(x < min, 0.0, np.where(x > max, 1.0, (x - min) / (max - min)))

early_stopping = EarlyStopping(
    monitor='val_loss',  # 
    patience=500,        # 
    verbose=1,          # 
    mode='min',         # 
    restore_best_weights=True  # 
)

2024-11-24 16:40:28.025098: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 16:40:28.071556: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 16:40:28.071578: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 16:40:28.071607: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 16:40:28.080330: I tensorflow/core/platform/cpu_feature_g

2.14.0
CPU times: user 2.77 s, sys: 2.35 s, total: 5.13 s
Wall time: 2.76 s


In [2]:
SAVE_DIR = "../data"
file_criteo = SAVE_DIR + "/criteo-uplift-v2.1.csv"
df_criteo_ori = pd.read_csv(file_criteo, sep=',')

In [3]:
df_criteo_ori.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
count,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07,1.397959e+07
mean,1.962030e+01,1.006998e+01,8.446582e+00,4.178923e+00,1.033884e+01,4.028513e+00,-4.155356e+00,5.101765e+00,3.933581e+00,1.602764e+01,5.333396e+00,-1.709672e-01,8.500001e-01,2.916680e-03,4.699200e-02,3.063122e-02
std,5.377464e+00,1.047557e-01,2.993161e-01,1.336645e+00,3.433081e-01,4.310974e-01,4.577914e+00,1.205248e+00,5.665958e-02,7.018975e+00,1.682288e-01,2.283277e-02,3.570713e-01,5.392748e-02,2.116217e-01,1.723164e-01
min,1.261636e+01,1.005965e+01,8.214383e+00,-8.398387e+00,1.028053e+01,-9.011892e+00,-3.142978e+01,4.833815e+00,3.635107e+00,1.319006e+01,5.300375e+00,-1.383941e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.261636e+01,1.005965e+01,8.214383e+00,4.679882e+00,1.028053e+01,4.115453e+00,-6.699321e+00,4.833815e+00,3.910792e+00,1.319006e+01,5.300375e+00,-1.686792e-01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.192341e+01,1.005965e+01,8.214383e+00,4.679882e+00,1.028053e+01,4.115453e+00,-2.411115e+00,4.833815e+00,3.971858e+00,1.319006e+01,5.300375e+00,-1.686792e-01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.443646e+01,1.005965e+01,8.723335e+00,4.679882e+00,1.028053e+01,4.115453e+00,2.944427e-01,4.833815e+00,3.971858e+00,1.319006e+01,5.300375e+00,-1.686792e-01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.674526e+01,1.634419e+01,9.051962e+00,4.679882e+00,2.112351e+01,4.115453e+00,2.944427e-01,1.199840e+01,3.971858e+00,7.529502e+01,6.473917e+00,-1.686792e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [4]:
%%time

sample = 1.0
random_state=20220720
df_criteo=df_criteo_ori.sample(frac=sample, random_state=random_state).reset_index(drop=True)
X = df_criteo[['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11']].values

X[:, 0] = scaling(X[:, 0], min=np.min(X[:, 0]), max=np.max(X[:, 0]))
X[:, 1] = scaling(X[:, 1], min=np.min(X[:, 1]), max=np.max(X[:, 1]))
X[:, 2] = scaling(X[:, 2], min=np.min(X[:, 2]), max=np.max(X[:, 2]))
X[:, 3] = scaling(X[:, 3], min=np.min(X[:, 3]), max=np.max(X[:, 3]))
X[:, 4] = scaling(X[:, 4], min=np.min(X[:, 4]), max=np.max(X[:, 4]))
X[:, 5] = scaling(X[:, 5], min=np.min(X[:, 5]), max=np.max(X[:, 5]))
X[:, 6] = scaling(X[:, 6], min=np.min(X[:, 6]), max=np.max(X[:, 6]))
X[:, 7] = scaling(X[:, 7], min=np.min(X[:, 7]), max=np.max(X[:, 7]))
X[:, 8] = scaling(X[:, 8], min=np.min(X[:, 8]), max=np.max(X[:, 8]))
X[:, 9] = scaling(X[:, 9], min=np.min(X[:, 9]), max=np.max(X[:, 9]))
X[:, 10] = scaling(X[:, 10], min=np.min(X[:, 10]), max=np.max(X[:, 10]))
X[:, 11] = scaling(X[:, 11], min=np.min(X[:, 11]), max=np.max(X[:, 11]))

T = df_criteo['treatment'].values.reshape(-1, 1)
Y_visit = df_criteo['visit'].values.reshape(-1, 1)
Y_conv = df_criteo['conversion'].values.reshape(-1, 1)

T.shape, Y_visit.shape, Y_conv.shape


# calculate len
train_len = int(len(X) * 0.70)
cali_len = int(len(X) * 0.05)
test_len = len(X) - train_len - cali_len

# obtain train set
X_train = X[:train_len, :]
T_train = T[:train_len, :]
Y_visit_train = Y_visit[:train_len, :]
Y_conv_train = Y_conv[:train_len, :]

# obtain calibration set
X_cali = X[train_len:train_len+cali_len, :]
T_cali = T[train_len:train_len+cali_len, :]
Y_visit_cali = Y_visit[train_len:train_len+cali_len, :]
Y_conv_cali = Y_conv[train_len:train_len+cali_len, :]

# obtain test set
X_test = X[train_len+cali_len:, :]
T_test = T[train_len+cali_len:, :]
Y_visit_test = Y_visit[train_len+cali_len:, :]
Y_conv_test = Y_conv[train_len+cali_len:, :]

print(train_len, X_train.shape, X_test.shape, len(X), X_cali.shape)

# make covariate shift
# condition_cali = (X_cali[:, 0] > 0.3) & (X_cali[:, 1] < 0.7)
# X_cali = X_cali[condition_cali]
# T_cali = T_cali[condition_cali]
# Y_visit_cali = Y_visit_cali[condition_cali]
# Y_conv_cali = Y_conv_cali[condition_cali]

# condition_test = (X_test[:, 0] > 0.3) & (X_test[:, 1] < 0.7)
# X_test = X_test[condition_test]
# T_test = T_test[condition_test]
# Y_visit_test = Y_visit_test[condition_test]
# Y_conv_test = Y_conv_test[condition_test]

# print(train_len, X_train.shape, X_test.shape, len(X), X_cali.shape)

9785714 (9785714, 12) (3494899, 12) 13979592 (698979, 12)
CPU times: user 2.98 s, sys: 1.19 s, total: 4.17 s
Wall time: 4.17 s


# First, on the train set:

### (i) Train DRP model

In [5]:
%%time
# DPR on train set
sys.path.append("..")
from model.uplift_model import *

count = 20
# 2.5e-5
import keras
import keras.backend as K
import tensorflow as tf
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from model.roi_model import *

# final_model = get_roi_rank_criteo_model()
# final_model.compile(loss=None, optimizer='adam')
#lambda y_true,y_pred: y_pred
# print('trainable_weights')
# for x in final_model.trainable_weights:
#     print(x.name)
# print('non_trainable_weights')
# for x in final_model.non_trainable_weights:
#     print(x.name)
# final_model.summary()

for i in range(count):

    
    
    final_model = get_roi_rank_criteo_model()
    final_model.compile(loss=None, optimizer='adam')

    mcp_save = ModelCheckpoint('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample), save_best_only=False, monitor='val_loss', mode='min', save_weights_only=True)
    history = final_model.fit([X_train, T_train, Y_conv_train, Y_visit_train], validation_split=0.2, epochs=10000, batch_size=10000000, shuffle=True, verbose=0, callbacks=[mcp_save, early_stopping])
    # plot_loss(history, "loss", "val_loss", "obj", "val_obj")

# Second, on the calibration set:

### (i) Infer DRP model to obtain $\hat{roi}$.

In [6]:
%%time
import sklearn 
import sklearn.metrics
from metric.Metric import *
sys.path.append("..")
from model.roi_model import *

count = 20

DRP_aucc_cali_list = []
roi_rank_pre_cali_list = []
for i in range(count):
    
    
    
    final_model = get_roi_rank_criteo_model()
    final_model.load_weights('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample))

    
    roi_rank_pre_cali = final_model.predict([X_cali, T_cali, Y_conv_cali, Y_visit_cali])
    DRP_aucc = get_uplift_model_aucc_no_show(t=(T_cali > 0.5).flatten(), y_reward=Y_conv_cali.flatten(), y_cost=Y_visit_cali.flatten(), roi_pred=roi_rank_pre_cali.flatten(), quantile=200)
    roi_rank_pre_cali_list.append(roi_rank_pre_cali)
    DRP_aucc_cali_list.append(DRP_aucc)
    
    

### (ii) Calculate $roi^*$ by Algorithm 2.

In [7]:
import numpy as np
from scipy.misc import derivative
from scipy.special import expit as sigmoid
from scipy.special import logit

def func(ti, yr, yc, si):
    qi = sigmoid(si)
    if ti == 1:
        return - yr*np.log(qi/(1-qi)) - yc*np.log(1-qi)
    else:
        return yr*np.log(qi/(1-qi)) + yc*np.log(1-qi)

    
def derivative_values(ti, yr, yc, si):
    def func_fixed_y(si_val):
        return func(ti, yr, yc, si_val)
    deriv = derivative(func_fixed_y, si, dx=1e-6)
    return deriv


def bst(ti, yr, yc):
    epsilon = 1e-5
    # corner case
    if yr == 0:
        return epsilon
    elif yc == 0:
        return 1 - epsilon
    
    roi_l = 0.0
    roi_r = 1.0
    roi_star = (roi_l + roi_r) / 2
    

    while np.abs(roi_r - roi_l) > epsilon:
        derivatives_at_si = derivative_values(ti, yr, yc, logit(roi_star))
        if np.abs(derivatives_at_si) < epsilon:
            break
        if derivatives_at_si > 0:
            roi_r = roi_star
        else:
            roi_l = roi_star
        roi_star = (roi_l + roi_r) / 2
    return roi_star


results = np.zeros(T_cali.shape)

# iterate
for i in range(T_cali.shape[0]):
    # call bst
    results[i] = bst(T_cali[i], Y_conv_cali[i], Y_visit_cali[i])

In [8]:
delta_roi = np.abs(results - roi_rank_pre_cali)

### (iii) Infer DRP's MC Dropout model to obtain $\hat{r}(x)$.

In [6]:
%%time
import sklearn 
import sklearn.metrics
from metric.Metric import *
from concurrent.futures import ProcessPoolExecutor

DRP_aucc_cali_mc_list = []

for i in range(count):
    
    print("iteration = ", i + 1)
    
    final_model = get_roi_rank_criteo_model_with_dropout()
    final_model.load_weights('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample))

    # multiple prediction
    n_iterations = 10
    all_predictions_cali = []
    
    for j in range(n_iterations):
        print(j)
        
        print("cali")
        predictions = final_model.predict([X_cali, T_cali, Y_conv_cali, Y_visit_cali])
        all_predictions_cali.append(predictions)
        DRP_aucc = get_uplift_model_aucc_no_show(t=(T_cali > 0.5).flatten(), y_reward=Y_conv_cali.flatten(), y_cost=Y_visit_cali.flatten(), roi_pred=predictions.flatten(), quantile=200)
        DRP_aucc_cali_list.append(DRP_aucc[0])
    
    print("\n")
    
# obtain mean and std
mean_pred = np.mean(all_predictions_cali, axis=0)
std_pred = np.std(all_predictions_cali, axis=0)


iteration =  1
0
cali
21844/21844 [==============================] - 23s 1ms/step
AUCC =  0.7575243712114315
1
cali
21844/21844 [==============================] - 23s 1ms/step
AUCC =  0.7462000213573595
2
cali
21844/21844 [==============================] - 22s 1ms/step
AUCC =  0.7569660093627335
3
cali
21844/21844 [==============================] - 23s 1ms/step
AUCC =  0.7601175616718252
4
cali
21844/21844 [==============================] - 23s 1ms/step
AUCC =  0.7756798618975613
5
cali
21844/21844 [==============================] - 22s 1ms/step
AUCC =  0.7372979483954843
6
cali
21844/21844 [==============================] - 22s 1ms/step
AUCC =  0.7615305739271512
7
cali
21844/21844 [==============================] - 23s 1ms/step
AUCC =  0.7589207735907034
8
cali
21844/21844 [==============================] - 23s 1ms/step
AUCC =  0.7690306381152133
9
cali
21844/21844 [==============================] - 22s 1ms/step
AUCC =  0.7765165413583895


CPU times: user 6min 42s, sys: 30.6 s, tota

### (iv) Calculate $\hat{q}$ by Algorithm 3

In [10]:
# Get scores
cal_scores = delta_roi/(std_pred + 1e-5)
n = len(cal_scores)

# set the error rate
alpha = 0.20
# Get the score quantile
qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation='higher')

### (v) Select $\widetilde{roi}$'s equation form from 5a to 5c.

In [13]:
# For mc only


ROI_1 = roi_rank_pre_cali*(4.666666*roi_rank_pre_cali + std_pred) # AUCC 
ROI_2 = roi_rank_pre_cali/(std_pred + 1e5)
ROI_3 = 4.666666*roi_rank_pre_cali + std_pred
DRP_aucc_cali = get_uplift_model_aucc_no_show(t=(T_cali > 0.5).flatten(), y_reward=Y_conv_cali.flatten(), y_cost=Y_visit_cali.flatten(), roi_pred=ROI_3.flatten(), quantile=200)

AUCC =  0.7761249185003556


In [ ]:
ROI_1 = roi_rank_pre_cali*(4.666666*roi_rank_pre_cali + qhat*std_pred) # AUCC 
ROI_2 = roi_rank_pre_cali/(qhat*std_pred + 1e5)
ROI_3 = (2.92*qhat*std_pred + qhat*std_pred) #
DRP_aucc_cali = get_uplift_model_aucc_no_show(t=(T_cali > 0.5).flatten(), y_reward=Y_conv_cali.flatten(), y_cost=Y_visit_cali.flatten(), roi_pred=ROI_1.flatten(), quantile=200)

# Third, on the test set:

### (i) Infer DRP model to obtain $\hat{roi}$.

In [14]:
import sklearn 
import sklearn.metrics
from metric.Metric import *

count = 20
sample = 1.0

DRP_aucc_test_list = []
roi_rank_pre_test_list = []
for i in range(count):
    
    
    final_model = get_roi_rank_criteo_model()
    final_model.load_weights('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample))

    roi_rank_pre_test = final_model.predict([X_test, T_test, Y_conv_test, Y_visit_test])
    roi_rank_pre_test_list.append(roi_rank_pre_test)
    DRP_aucc = get_uplift_model_aucc_no_show(t=(T_test > 0.5).flatten(), y_reward=Y_conv_test.flatten(), y_cost=Y_visit_test.flatten(), roi_pred=roi_rank_pre_test.flatten(), quantile=200)
    DRP_aucc_test_list.append(DRP_aucc)
    
    
mean_pred = np.mean(DRP_aucc_test_list, axis=0)
std_pred = np.std(DRP_aucc_test_list, axis=0)



print(f"mean = {mean_pred:.4f}")
print(f"std = {std_pred:.4f}")



mean = 0.7714
std = 0.0009


In [9]:
# store test aucc for pic 
import pandas as pd

def get_aucc_cost_curve(aucc_list):
    delta_cost_list_group = np.array([aucc[1] for aucc in aucc_list])
    delta_reward_list_group = np.array([aucc[2] for aucc in aucc_list])
    
    avg_delta_cost_list = np.mean(delta_cost_list_group, axis=0)
    avg_delta_reward_list = np.mean(delta_reward_list_group, axis=0)
    
    df_aucc_cost_curve = pd.DataFrame(avg_delta_cost_list, columns=['delta_cost'])
    df_aucc_cost_curve['delta_reward'] = avg_delta_reward_list
    
    return df_aucc_cost_curve

DRP_avg_aucc_cost_curve = get_aucc_cost_curve(DRP_aucc_test_list)
print("aucc = ", np.sum(DRP_avg_aucc_cost_curve['delta_reward'].values) / (DRP_avg_aucc_cost_curve['delta_reward'].values[-1] * 201))
DRP_avg_aucc_cost_curve.to_csv("../figure/a_DRP_avg_aucc_cost_curve_1.0.csv")

aucc =  0.7714120677124195


### (ii) Infer DRP's MC Dropout to obtain $\hat{r}(x)$.

In [8]:
%%time
import time
import sklearn 
import sklearn.metrics
from metric.Metric import *
from concurrent.futures import ProcessPoolExecutor
sys.path.append("..")
from model.uplift_model import *
from model.roi_model import *


count = 20
sample = 1.0

DRP_aucc_test_mc_list = []

for i in range(count):
    
    print("iteration = ", i + 1)
    
    final_model = get_roi_rank_criteo_model_with_dropout()
    final_model.load_weights('../model_file/roi/criteo/final_model/roi_rank/roi_rank_criteo_model_{}_{}.h5'.format(i+1, sample))

    # multiple prediction
    n_iterations = 10
    all_predictions_test = []
    
    for j in range(n_iterations):
        print(j)
        
        print("test")
        predictions = final_model.predict([X_test, T_test, Y_conv_test, Y_visit_test])
        all_predictions_test.append(predictions)
        DRP_aucc_mc = get_uplift_model_aucc_no_show(t=(T_test > 0.5).flatten(), y_reward=Y_conv_test.flatten(), y_cost=Y_visit_test.flatten(), roi_pred=predictions.flatten(), quantile=200)
        DRP_aucc_test_mc_list.append(DRP_aucc_mc[0])
    
    print("\n")
    
# cal std and mean
mean_pred_test = np.mean(all_predictions_test, axis=0)
std_pred_test = np.std(all_predictions_test, axis=0)

end_time = time.time()
print(end_time)

iteration =  1
0
test
109216/109216 [==============================] - 113s 1ms/step
AUCC =  0.7580269187877774
1
test
109216/109216 [==============================] - 113s 1ms/step
AUCC =  0.7607220519330183
2
test
109216/109216 [==============================] - 113s 1ms/step
AUCC =  0.7634322691232226
3
test
109216/109216 [==============================] - 114s 1ms/step
AUCC =  0.7622789776120242
4
test
109216/109216 [==============================] - 113s 1ms/step
AUCC =  0.7568553781887694
5
test
109216/109216 [==============================] - 113s 1ms/step
AUCC =  0.7544438167891183
6
test
109216/109216 [==============================] - 112s 1ms/step
AUCC =  0.7691626496152824
7
test
109216/109216 [==============================] - 112s 1ms/step
AUCC =  0.7612667821349454
8
test
109216/109216 [==============================] - 113s 1ms/step
AUCC =  0.7574737330217997
9
test
109216/109216 [==============================] - 113s 1ms/step
AUCC =  0.7605958300709663


1701538920.72

### (iii) With $\hat{q}$  and the selected expression, $\widetilde{roi}(x_{test})$ is obtained.

In [16]:
calculate rDRP aucc
rDRP_aucc_test_list = []
ROI_1 = roi_rank_pre_test*(4.666666*roi_rank_pre_test + qhat*std_pred_test) # 
rDRP_aucc = get_uplift_model_aucc_no_show(t=(T_test > 0.5).flatten(), y_reward=Y_conv_test.flatten(), y_cost=Y_visit_test.flatten(), roi_pred=ROI_1.flatten(), quantile=200)
rDRP_aucc_test_list.append(rDRP_aucc)


mean_pred = np.mean(rDRP_aucc_test_list, axis=0)
std_pred = np.std(rDRP_aucc_test_list, axis=0)



print(f"mean = {mean_pred:.4f}")
print(f"std = {std_pred:.4f}")




# # save rDRP AUCC result

# rDRP_avg_aucc_cost_curve = get_aucc_cost_curve(rDRP_aucc_test_list)
# print("aucc = ", np.sum(rDRP_avg_aucc_cost_curve['delta_reward'].values) / (rDRP_avg_aucc_cost_curve['delta_reward'].values[-1] * 201))
# rDRP_avg_aucc_cost_curve.to_csv("../figure/rDRP_avg_aucc_cost_curve_1.0.csv")

mean = 0.7717
std = 0.0007
